In [ ]:
import itk
import os
import ipywidgets as widgets
import math
import numpy as np
import shutil
import xlrd
import itk
import os
import math
import numpy as np
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
pio.orca.config.use_xvfb = True
import statistics
from statistics import mean
import xlsxwriter
import os 
from os.path import join as pjoin
from openpyxl import Workbook
import numpy as np
import matplotlib as plt
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import xlsxwriter
import os 
from os.path import join as pjoin
import scipy.interpolate as spi
from openpyxl import Workbook
import pandas as pd
import gatetools as gt

def ComputeSize(point, matrix, returnMin):
    #Define extreme corner points
    points = [itk.Vector.D3(0.0) for i in range(8)]
    points[1].SetElement(0, point.GetElement(0))
    points[2].SetElement(1, point.GetElement(1))
    points[3].SetElement(2, point.GetElement(2))
    points[4].SetElement(0, point.GetElement(0))
    points[4].SetElement(1, point.GetElement(1))
    points[5].SetElement(1, point.GetElement(1))
    points[5].SetElement(2, point.GetElement(2))
    points[6].SetElement(0, point.GetElement(0))
    points[6].SetElement(2, point.GetElement(2))
    points[7].SetElement(0, point.GetElement(0))
    points[7].SetElement(1, point.GetElement(1))
    points[7].SetElement(2, point.GetElement(2))

    #Transform cornes
    for i in range(len(points)):
        points[i] = matrix*points[i]

    #Find min and max
    minBB = itk.Vector.D3(0.0)
    maxBB = itk.Vector.D3(0.0)
    for i in range(0,3):
        minBB.SetElement(i, min([points[j].GetElement(i) for j in range(len(points))]))
        maxBB.SetElement(i, max([points[j].GetElement(i) for j in range(len(points))]))

    if returnMin:
        return(minBB)
    else:
        return(maxBB-minBB)

def TransformImage(image, transform):
    #Compute output characteristics (size, origin, spacing, ...)
    invTransform = itk.Euler3DTransform[itk.D].New()
    transform.GetInverse(invTransform)
    outputOrigin = invTransform.GetMatrix()*image.GetOrigin()+invTransform.GetTranslation()
    inputSize = itk.Vector.D3(0.0)
    inputSizemm = itk.Vector.D3(0.0)
    outputSpacing = itk.Vector.D3(0.0)
    for i in range(len(image.GetLargestPossibleRegion().GetSize())):
        inputSize[i] = image.GetLargestPossibleRegion().GetSize()[i]
        inputSizemm[i] = image.GetLargestPossibleRegion().GetSize()[i]*image.GetSpacing()[i]
    outputSize = ComputeSize(inputSize, invTransform.GetMatrix()*image.GetDirection(), 0)
    outputSizemm = ComputeSize(inputSizemm, invTransform.GetMatrix()*image.GetDirection(), 0)
    outputOffset = ComputeSize(inputSizemm, invTransform.GetMatrix()*image.GetDirection(), 1)
    for i in range(3):
        outputSpacing[i] = outputSizemm[i]/round(outputSize[i])
    outputOrigin += outputOffset
    realOutputSize = image.GetLargestPossibleRegion().GetSize()
    for i in range(3):
        if outputSize[i] < 0:
            outputSize[i] *= -1
            outputOrigin[i] += outputSpacing[i] * (outputSize[i]-1)
            outputSpacing[i] *= -1
        realOutputSize[i] = round(outputSize[i])

    #Create the resample filter
    resampleFilter = itk.ResampleImageFilter.New(image)
    resampleFilter.SetTransform(transform)
    resampleFilter.UseReferenceImageOff()
    resampleFilter.SetSize(realOutputSize)
    resampleFilter.SetOutputSpacing(outputSpacing)
    resampleFilter.SetOutputOrigin(outputOrigin)
    resampleFilter.SetDefaultPixelValue(0)
    resampleFilter.Update()
    return resampleFilter.GetOutput()
    

class FileBrowser(object):
    def __init__(self, querringType=None, fileType = None):
        self.path = os.getcwd()
        self.querringFolder = 0
        self.fileType = fileType
        if querringType == "folder":
            self.querringFolder = 1
        self._update_files()
        
    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = os.path.join(self.path, f)
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    if not self.querringFolder:
                        if self.fileType == None or os.path.splitext(f)[1] in self.fileType:
                            self.files.append(f)
        
    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box
    
    def _update(self, box):
        
        def on_click(b):
            if b.description == '..':
                self.path = os.path.split(self.path)[0]
            else:
                self.path = os.path.join(self.path, b.description)
            self._update_files()
            self._update(box)
        
        buttons = []
        button = widgets.Button(description='..', background_color='#d0d0ff')
        button.on_click(on_click)
        buttons.append(button)
        for f in self.dirs:
            button = widgets.Button(description=f, background_color='#d0d0ff', width='400px', height='50px')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.files:
            button = widgets.Button(description=f, width='400px', height='50px')
            button.on_click(on_click)
            buttons.append(button)
        box.children = tuple([widgets.HTML("<h2>%s</h2>" % (self.path,))] + buttons)

monacoDoseFiles = []
patientSelection = "C:/Users/DIFRANCF/Desktop/TemplateMU/TEMPmu2/Chalmey0mha"
displacementSelections = []
for r, d, f in os.walk(patientSelection):
    for file in f:
        if file.startswith("2009466_Chalmey.mhd"):
            monacoDoseFiles.append(os.path.join(r, file))
        if file.endswith(".xls") and "Results_" not in file:
            displacementSelections.append(os.path.join(r, file))


for displacementSelection in displacementSelections:
    
    seanceName = displacementSelection[:-5]
    print(displacementSelection)

    loc = (displacementSelection) 
    wb = xlrd.open_workbook(loc) 
    LR = wb.sheet_by_index(0) 
    SI = wb.sheet_by_index(1) 
    AP = wb.sheet_by_index(2) 


    print(LR.nrows-3)
    
    val = LR.nrows-3

    #Data recovery from the 3 excel sheetsC:/Users/DIFRANCF/Desktop/    val = LR./rows-3 #Nume/o di casel/e in cui sono presenti I deplacements 18 19

    #Copy of the mean values each 10s in each direction within a matrix (One patient only)

    mx = np.zeros((val, 1)) #mx = np.zeros((1, val))   
    my = np.zeros((val, 1)) #my = np.zeros((1, val)) 
    mz = np.zeros((val, 1)) #mz = np.zeros((1, val))  
    l=3 # l=2
    c=1 #c=3

    for i in range(val): #1
            mx[i, 0] = LR.cell_value(l, c)
            my[i, 0] = SI.cell_value(l, c)
            mz[i, 0] = AP.cell_value(l, c)
            l=l+1 #c = c+1

    #Recover the displacement values 

    l = 0
    x = np.zeros(val)
    y = np.zeros(val)
    z = np.zeros(val)
    patient = os.path.basename(patientSelection)

    while LR.cell_value(l,1) != patient:   #(l,0)
        l=l+1
    for j in range(val):
    #     x[j] = mx[l-2, j]
    #     y[j] = -my[l-2, j]
    #     z[j] = mz[l-2, j]
            x[j] = mx[j, l]
            y[j] = -my[j, l]
            z[j] = mz[j, l]


    monacoInputDose = itk.imread(monacoDoseFiles[0])
    if os.path.isdir(seanceName):
        shutil.rmtree(seanceName)

    os.mkdir(seanceName)

    #Display progression bar
    progressionBar = widgets.IntProgress(
        value=0,
        min=0,
        max = len(x),
        #max=nbDisplacement,
        step=1,
        description='Progression:',
        bar_style='info',
        orientation='horizontal'
    )
    display(progressionBar)

    for i in range(len(x)):
        #for i in range(nbDisplacement):
        #Create the transformation, convert degree in radian
        transform = itk.Euler3DTransform[itk.D].New()
        parameters = transform.GetParameters()
        parameters[3] = x[i]
        parameters[4] = y[i]
        parameters[5] = z[i]

        transform.SetParameters(parameters)


        #Transform the 2 dose images

        movedMonacoDose = gt.applyTransformation(input=monacoInputDose, translation=[x[i], y[i], z[i]], pad=0, force_resample=True, interpolation_mode='linear')


        monacoPath = os.path.join(seanceName, "t" + f'{i:03d}' +"_S1")



        if os.path.isdir(monacoPath):
            shutil.rmtree(monacoPath)

        os.mkdir(monacoPath)

        #Write the outputs
        itk.imwrite(movedMonacoDose, os.path.join(monacoPath, "dose.mhd"))

        #Update progress bar
        progressionBar.value += 1

